In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

pd.set_option("display.max_columns", None)

C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
betting_df = pd.read_excel('https://www.aussportsbetting.com/historical_data/nfl.xlsx')

In [3]:
betting_df.head()

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,Home Line Open,Home Line Min,Home Line Max,Home Line Close,Away Line Open,Away Line Min,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,Total Score Open,Total Score Min,Total Score Max,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2023-02-12 00:00:00,Philadelphia Eagles,Kansas City Chiefs,35,38,NaN,Y,Y,1.833,1.740,1.833,1.833,2.000,2.000,2.15,2.00,-1.5,-2.5,-1.0,-1.0,1.5,1.0,2.5,1.0,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,49.5,49.5,51.5,51.5,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,NaN
1,2023-01-29 00:00:00,Kansas City Chiefs,Cincinnati Bengals,23,20,NaN,Y,NaN,1.869,1.769,2.200,1.769,1.952,1.714,2.10,2.10,-1.0,-2.0,2.5,-2.0,1.0,-2.5,2.0,2.0,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,47.5,46.5,48.5,48.5,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,NaN
2,2023-01-29 00:00:00,Philadelphia Eagles,San Francisco 49ers,31,7,NaN,Y,NaN,1.769,1.645,1.769,1.645,2.100,2.100,2.35,2.35,-1.5,-3.0,-1.5,-3.0,1.5,1.5,3.0,3.0,1.869,2.000,1.869,2.000,1.952,1.952,1.833,1.833,45.5,45.0,46.5,45.0,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,NaN
3,2023-01-22 00:00:00,San Francisco 49ers,Dallas Cowboys,19,12,NaN,Y,NaN,1.500,1.476,1.540,1.500,2.700,2.650,2.75,2.70,-4.0,-4.0,-3.5,-3.5,4.0,3.5,4.0,3.5,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,46.0,45.5,47.0,46.5,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,NaN
4,2023-01-22 00:00:00,Buffalo Bills,Cincinnati Bengals,10,27,NaN,Y,NaN,1.520,1.384,1.520,1.384,2.670,2.670,3.15,3.15,-4.0,-6.0,-4.0,-6.0,4.0,4.0,6.0,6.0,1.952,1.909,1.952,1.909,1.869,1.869,1.909,1.909,50.0,48.0,50.5,48.5,1.909,1.909,1.909,1.909,1.909,1.909,1.909,1.909,NaN


In [4]:
betting_data = betting_df.copy()
betting_data = betting_data[['Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Home Odds Open', 'Away Odds Open',
                             'Home Line Open', 'Away Line Open', 'Total Score Open','Home Odds Close', 'Away Odds Close', 
                             'Home Line Close', 'Away Line Close', 'Total Score Close']]

In [5]:
def decimal_to_moneyline(decimal):
    if np.isnan(decimal):
        return np.nan
    elif decimal >= 2.00:
        return np.round((decimal-1) * 100)
    else: 
        return np.round(-100 / (decimal-1))
    
def get_over_under_result(game, lineopen=True):
    game_score = game['Total Score']
    if lineopen:
        ou_score = game['Total Score Open']
    else:
        ou_score = game['Total Score Close']
        
    if game_score == ou_score :
        result = 'push'
    elif game_score > ou_score:
        result = 'over'
    else:
        result = 'under'
        
    return result 

def fix_home_team_names(game):
    team_mapping = {
        'Oakland Raiders':'Las Vegas Raiders',
        'St. Louis Rams':'Los Angeles Rams',
        'San Diego Chargers':'Los Angeles Chargers',
        'Washington Redskins':'Washington Football Team',
        'Washington Commanders':'Washington Football Team'
    }
    
    if game['Home Team'] in team_mapping.keys():
        return team_mapping[game['Home Team']]
    
    else:
        return game['Home Team']
    
def fix_away_team_names(game):
    team_mapping = {
        'Oakland Raiders':'Las Vegas Raiders',
        'St. Louis Rams':'Los Angeles Rams',
        'San Diego Chargers':'Los Angeles Chargers',
        'Washington Redskins':'Washington Football Team',
        'Washington Commanders':'Washington Football Team'
    }
    
    if game['Away Team'] in team_mapping.keys():
        return team_mapping[game['Away Team']]
    
    else:
        return game['Away Team']
    
def get_spread_cover_team(game, lineopen = True):
    if lineopen:
        if game['Home Team Actual Line'] < game['Home Line Open']:
            return 'home'
        elif game['Home Team Actual Line'] > game['Home Line Open']:
            return 'away'
        elif game['Home Team Actual Line'] == game['Home Line Open']:
            return 'push'
        
    else:
        if game['Home Team Actual Line'] < game['Home Line Close']:
            return 'home'
        elif game['Home Team Actual Line'] > game['Home Line Close']:
            return 'away'
        elif game['Home Team Actual Line'] == game['Home Line Close']:
            return 'push'
        
        
def get_winner(game, lineopen = True):
    if game['Home Score'] > game['Away Score']:
        return 'home'
    elif game['Home Score'] < game['Away Score']:
        return 'away'
    elif game['Home Score'] == game['Away Score']:
        return 'push'


betting_data['Home Team'] = betting_data.apply(lambda x: fix_home_team_names(x), axis=1)
betting_data['Away Team'] = betting_data.apply(lambda x: fix_away_team_names(x), axis=1)

betting_data['Home ML Open'] = betting_data['Home Odds Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away ML Open'] = betting_data['Away Odds Open'].apply(lambda x: decimal_to_moneyline(x))

betting_data['Home ML Close'] = betting_data['Home Odds Close'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away ML Close'] = betting_data['Away Odds Close'].apply(lambda x: decimal_to_moneyline(x))

betting_data['Total Score'] = betting_data.apply(lambda x: x['Home Score'] + x['Away Score'], axis=1)

betting_data['Over/Under Open Result'] = betting_data.apply(lambda x: get_over_under_result(x, lineopen=True), axis=1)
betting_data['Over/Under Close Result'] = betting_data.apply(lambda x: get_over_under_result(x, lineopen=False), axis=1)

betting_data['Home Team Actual Line'] = betting_data.apply(lambda x: x['Away Score'] - x['Home Score'], axis=1)
betting_data['Away Team Actual Line'] = betting_data.apply(lambda x: x['Home Score'] - x['Away Score'], axis=1)

betting_data['Spread Cover Open Result'] = betting_data.apply(lambda x: get_spread_cover_team(x, lineopen=True), axis=1)
betting_data['Spread Cover Close Result'] = betting_data.apply(lambda x: get_spread_cover_team(x, lineopen=False), axis=1)

betting_data['Game Winner'] = betting_data.apply(lambda x: get_winner(x, lineopen=True), axis=1)

betting_data.head(5)

,Date,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Away Odds Open,Home Line Open,Away Line Open,Total Score Open,Home Odds Close,Away Odds Close,Home Line Close,Away Line Close,Total Score Close,Home ML Open,Away ML Open,Home ML Close,Away ML Close,Total Score,Over/Under Open Result,Over/Under Close Result,Home Team Actual Line,Away Team Actual Line,Spread Cover Open Result,Spread Cover Close Result,Game Winner
0,2023-02-12 00:00:00,Philadelphia Eagles,Kansas City Chiefs,35,38,1.833,2.000,-1.5,1.5,49.5,1.833,2.00,-1.0,1.0,51.5,-120.0,100.0,-120.0,100.0,73,over,over,3,-3,away,away,away
1,2023-01-29 00:00:00,Kansas City Chiefs,Cincinnati Bengals,23,20,1.869,1.952,-1.0,1.0,47.5,1.769,2.10,-2.0,2.0,48.5,-115.0,-105.0,-130.0,110.0,43,under,under,-3,3,home,home,home
2,2023-01-29 00:00:00,Philadelphia Eagles,San Francisco 49ers,31,7,1.769,2.100,-1.5,1.5,45.5,1.645,2.35,-3.0,3.0,45.0,-130.0,110.0,-155.0,135.0,38,under,under,-24,24,home,home,home
3,2023-01-22 00:00:00,San Francisco 49ers,Dallas Cowboys,19,12,1.500,2.700,-4.0,4.0,46.0,1.500,2.70,-3.5,3.5,46.5,-200.0,170.0,-200.0,170.0,31,under,under,-7,7,home,home,home
4,2023-01-22 00:00:00,Buffalo Bills,Cincinnati Bengals,10,27,1.520,2.670,-4.0,4.0,50.0,1.384,3.15,-6.0,6.0,48.5,-192.0,167.0,-260.0,215.0,37,under,under,17,-17,away,away,away


In [6]:
# Found a problem in the data set where the favorite and underdog teams don't match based on how they 
# should. Example, Jets vs. Packers on 12-23-2018 has the Jets as a 2.5 point underdog when looking at the spread, but a 
# -125 moneyline favorite. Assuming this is a data error, because a team shouldn't be a moneyline favorite but a point 
# spread underdog. 

# Removing any instances where the moneyline and spread favorites/underdogs don't match

betting_data = betting_data[((betting_data['Home Line Open'] < 0) & (betting_data['Home ML Open'] <= betting_data['Away ML Open'])) | 
            ((betting_data['Home Line Open'] > 0) & (betting_data['Home ML Open'] >= betting_data['Away ML Open'])) |
            (betting_data['Home Line Open'] == 0)]

In [7]:
# Rename Columns

renamed_betting_df = betting_data.copy()

renamed_betting_df = renamed_betting_df.rename(columns = {
    'Date': 'date',
    'Home Team': 'home_team',
    'Away Team': 'away_team',
    'Home Score': 'home_score',
    'Away Score': 'away_score',
    'Home Odds Open': 'home_odds_open',
    'Away Odds Open': 'away_odds_open',
    'Home Line Open': 'home_spread_open',
    'Away Line Open': 'away_spread_open',
    'Total Score Open': 'total_score_open',
    'Home Odds Close': 'home_odds_close',
    'Away Odds Close': 'away_odds_close',
    'Home Line Close': 'home_spread_close',
    'Away Line Close': 'away_spread_close',
    'Total Score Close': 'total_score_close',
    'Home ML Open': 'home_ml_open',
    'Away ML Open': 'away_ml_open',
    'Home ML Close': 'home_ml_close',
    'Away ML Close': 'away_ml_close',
    'Total Score': 'total_score_actual',
    'Over/Under Open Result': 'over_under_open_result',
    'Over/Under Close Result': 'over_under_close_result',
    'Home Team Actual Line': 'home_team_actual_line',
    'Away Team Actual Line': 'away_team_actual_line',
    'Spread Cover Open Result': 'spread_cover_open_result',
    'Spread Cover Close Result': 'spread_cover_close_result',
    'Game Winner': 'game_winner'
})

renamed_betting_df.head()

,date,home_team,away_team,home_score,away_score,home_odds_open,away_odds_open,home_spread_open,away_spread_open,total_score_open,home_odds_close,away_odds_close,home_spread_close,away_spread_close,total_score_close,home_ml_open,away_ml_open,home_ml_close,away_ml_close,total_score_actual,over_under_open_result,over_under_close_result,home_team_actual_line,away_team_actual_line,spread_cover_open_result,spread_cover_close_result,game_winner
0,2023-02-12 00:00:00,Philadelphia Eagles,Kansas City Chiefs,35,38,1.833,2.000,-1.5,1.5,49.5,1.833,2.00,-1.0,1.0,51.5,-120.0,100.0,-120.0,100.0,73,over,over,3,-3,away,away,away
1,2023-01-29 00:00:00,Kansas City Chiefs,Cincinnati Bengals,23,20,1.869,1.952,-1.0,1.0,47.5,1.769,2.10,-2.0,2.0,48.5,-115.0,-105.0,-130.0,110.0,43,under,under,-3,3,home,home,home
2,2023-01-29 00:00:00,Philadelphia Eagles,San Francisco 49ers,31,7,1.769,2.100,-1.5,1.5,45.5,1.645,2.35,-3.0,3.0,45.0,-130.0,110.0,-155.0,135.0,38,under,under,-24,24,home,home,home
3,2023-01-22 00:00:00,San Francisco 49ers,Dallas Cowboys,19,12,1.500,2.700,-4.0,4.0,46.0,1.500,2.70,-3.5,3.5,46.5,-200.0,170.0,-200.0,170.0,31,under,under,-7,7,home,home,home
4,2023-01-22 00:00:00,Buffalo Bills,Cincinnati Bengals,10,27,1.520,2.670,-4.0,4.0,50.0,1.384,3.15,-6.0,6.0,48.5,-192.0,167.0,-260.0,215.0,37,under,under,17,-17,away,away,away


In [8]:
# Get implied probability

def get_implied_probability(odds):
    if odds > 0:
        return (100/(odds + 100))
    else:
        return (-odds/(-odds + 100))

renamed_betting_df['home_implied_probability_open'] = renamed_betting_df.apply(lambda x: get_implied_probability(x.home_ml_open), axis=1)
renamed_betting_df['away_implied_probability_open'] = renamed_betting_df.apply(lambda x: get_implied_probability(x.away_ml_open), axis=1)
renamed_betting_df['home_implied_probability_close'] = renamed_betting_df.apply(lambda x: get_implied_probability(x.home_ml_close), axis=1)
renamed_betting_df['away_implied_probability_close'] = renamed_betting_df.apply(lambda x: get_implied_probability(x.away_ml_close), axis=1)

In [13]:
renamed_betting_df['date'] = pd.to_datetime(renamed_betting_df['date']).dt.date

In [14]:
# Truncated betting df with opening lines

opening_betting = renamed_betting_df.copy()
opening_betting = opening_betting[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'total_score_actual', 
                                  'home_ml_open', 'away_ml_open', 'home_spread_open', 'away_spread_open',
                                  'total_score_open', 'over_under_open_result', 'home_team_actual_line',
                                  'away_team_actual_line', 'spread_cover_open_result', 
                                   'home_implied_probability_open', 'away_implied_probability_open',
                                   'game_winner']]

opening_betting = opening_betting.rename(columns = {
    'home_ml_open': 'home_moneyline',
    'away_ml_open': 'away_moneyline',
    'home_spread_open': 'home_spread',
    'away_spread_open': 'away_spread',
    'total_score_open': 'total_score_line',
    'over_under_open_result': 'over_under_result',
    'spread_cover_open_result': 'spread_cover_result',
    'home_implied_probability_open': 'home_implied_prob',
    'away_implied_probability_open': 'away_implied_prob'
})

opening_betting.to_csv('../data/truncated_cleaned_betting_data_opening_lines.csv')
opening_betting.head()

,date,home_team,away_team,home_score,away_score,total_score_actual,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,home_implied_prob,away_implied_prob,game_winner
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,73,-120.0,100.0,-1.5,1.5,49.5,over,3,-3,away,0.545455,0.500000,away
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,43,-115.0,-105.0,-1.0,1.0,47.5,under,-3,3,home,0.534884,0.512195,home
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,38,-130.0,110.0,-1.5,1.5,45.5,under,-24,24,home,0.565217,0.476190,home
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,31,-200.0,170.0,-4.0,4.0,46.0,under,-7,7,home,0.666667,0.370370,home
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,37,-192.0,167.0,-4.0,4.0,50.0,under,17,-17,away,0.657534,0.374532,away


In [15]:
# Truncated betting df with closing lines

closing_betting = renamed_betting_df.copy()
closing_betting = closing_betting[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'total_score_actual', 
                                  'home_ml_close', 'away_ml_close', 'home_spread_close', 'away_spread_close',
                                  'total_score_close', 'over_under_close_result', 'home_team_actual_line',
                                  'away_team_actual_line', 'spread_cover_close_result', 
                                   'home_implied_probability_close', 'away_implied_probability_close',
                                   'game_winner']]

closing_betting = closing_betting.rename(columns = {
    'home_ml_close': 'home_moneyline',
    'away_ml_close': 'away_moneyline',
    'home_spread_close': 'home_spread',
    'away_spread_close': 'away_spread',
    'total_score_close': 'total_score_line',
    'over_under_close_result': 'over_under_result',
    'spread_cover_close_result': 'spread_cover_result',
    'home_implied_probability_close': 'home_implied_prob',
    'away_implied_probability_close': 'away_implied_prob'
})

closing_betting.to_csv('../data/truncated_cleaned_betting_data_closing_lines.csv')
closing_betting.head()

,date,home_team,away_team,home_score,away_score,total_score_actual,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,home_implied_prob,away_implied_prob,game_winner
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,73,-120.0,100.0,-1.0,1.0,51.5,over,3,-3,away,0.545455,0.500000,away
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,43,-130.0,110.0,-2.0,2.0,48.5,under,-3,3,home,0.565217,0.476190,home
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,38,-155.0,135.0,-3.0,3.0,45.0,under,-24,24,home,0.607843,0.425532,home
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,31,-200.0,170.0,-3.5,3.5,46.5,under,-7,7,home,0.666667,0.370370,home
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,37,-260.0,215.0,-6.0,6.0,48.5,under,17,-17,away,0.722222,0.317460,away
